In [1]:
include("../src/PhasorNetworks.jl")
using .PhasorNetworks, Plots

In [2]:
using Lux, MLUtils, MLDatasets, OneHotArrays, Statistics, Test
using Random: Xoshiro, AbstractRNG
using Base: @kwdef
using Zygote: withgradient
using LuxDeviceUtils: cpu_device, gpu_device
using Optimisers, ComponentArrays
using Statistics: mean
using LinearAlgebra: diag
using Distributions: Normal
using DifferentialEquations: Heun, Tsit5

In [3]:
@info "Running similarity test..."
function check_phase(matrix)
    in_phase = diag(matrix)
    anti_phase = diag(matrix, convert(Int, round(n_x / 2)))

    v1 = reduce(*, map(x -> x > 1.0 - epsilon, in_phase))
    v2 = reduce(*, map(x -> x < -1.0 + epsilon, anti_phase))
    return v1, v2
end

[ Info: Running similarity test...


check_phase (generic function with 1 method)

In [4]:
using DifferentialEquations

In [5]:
n_x = 101
n_y = 101
n_vsa = 1
epsilon = 0.10
repeats = 10
epsilon = 0.025
solver_args = Dict(:adaptive => true, :abstol => 1e-6, :reltol => 1e-6)
spk_args = SpikingArgs(t_window = 0.01, 
                    threshold = 0.001,
                    solver=Tsit5(), 
                    solver_args = solver_args)
tspan = (0.0, repeats*1.0)
tbase = collect(tspan[1]:spk_args.dt:tspan[2])

1001-element Vector{Float64}:
  0.0
  0.01
  0.02
  0.03
  0.04
  0.05
  0.06
  0.07
  0.08
  0.09
  0.1
  0.11
  0.12
  ⋮
  9.89
  9.9
  9.91
  9.92
  9.93
  9.94
  9.95
  9.96
  9.97
  9.98
  9.99
 10.0

In [6]:
phase_x = reshape(range(-1.0, 1.0, n_x), (1, n_x, n_vsa)) |> collect
phase_y = reshape(range(-1.0, 1.0, n_y), (1, n_y, n_vsa)) |> collect
sims = similarity_outer(phase_x, phase_y, dims= 2, reduce_dim=1)[1,1,:,:]
v1, v2 = check_phase(sims)

(true, true)

In [7]:
st_x = phase_to_train(phase_x, spk_args = spk_args, repeats = repeats)
st_y = phase_to_train(phase_y, spk_args = spk_args, repeats = repeats)
sims_2 = stack(similarity_outer(st_x, st_y, dims=2, reduce_dim=3, tspan=tspan, spk_args = spk_args));

LoadError: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 10 and 1010